In [1]:
import pandas as pd
import numpy as np
import math
import re
import time
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, accuracy
from surprise import BaselineOnly, KNNBasic, NormalPredictor 
from surprise.model_selection import KFold


In [2]:
start = time.process_time()
# Skip date
df1 = pd.read_csv('./combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
df2 = pd.read_csv('./combined_data_2.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
df3 = pd.read_csv('./combined_data_3.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
df4 = pd.read_csv('./combined_data_4.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

df1['Rating'] = df1['Rating'].astype(float)
df2['Rating'] = df2['Rating'].astype(float)
df3['Rating'] = df3['Rating'].astype(float)
df4['Rating'] = df4['Rating'].astype(float)

df = df1
df = df1.append(df2)
df = df.append(df3)
df = df.append(df4)


df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

# Data cleaning
# Movie ID is really a mess import! Looping through dataframe to add Movie ID column WILL make the 
# Kernel run out of memory as it is too inefficient. I achieve my task by first creating a numpy 
# array with correct length then add the whole array as column into the main dataframe! Let's see 
# how it is done below:
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))

# remove those Movie ID rows
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.iloc[::5000000, :])

# Data slicing
# The data set now is super huge. I have tried many different ways but can't get the Kernel running 
# as intended without memory error. Therefore I tried to reduce the data volumn by improving the 
# data quality below:

#     Remove movie with too less reviews (they are relatively not popular)
#     Remove customer who give too less reviews (they are relatively less active)

# Having above benchmark will have significant improvement on efficiency, since those unpopular movies 
# and non-active customers still occupy same volumn as those popular movies and active customers in the 
# view of matrix (NaN still occupy space). This should help improve the statistical signifiance too.

# Let's see how it is implemented:

sample_data=df.sample(frac=0.001, random_state=1)
print('After Samole Shape: {}'.format(sample_data.shape))

f = ['count','mean']
df_movie_summary = sample_data.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = sample_data.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

print('Original Shape: {}'.format(sample_data.shape))
sample_data = sample_data[~sample_data['Movie_Id'].isin(drop_movie_list)]
sample_data = sample_data[~sample_data['Cust_Id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(sample_data.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])



reader = Reader()
data = Dataset.load_from_df(sample_data[['Cust_Id', 'Movie_Id', 'Rating']], reader)
#data.split(n_folds=3)
train_set = data.build_full_trainset()

end = time.process_time()
print("运算时间为"+str(end-start))

Full dataset shape: (100498277, 2)
-Dataset examples-
           Cust_Id  Rating
0               1:     NaN
5000000    2560324     4.0
10000000   2271935     2.0
15000000   1921803     2.0
20000000   1933327     3.0
25000000   1465002     3.0
30000000    961023     4.0
35000000   1372532     5.0
40000000    854274     5.0
45000000    116334     3.0
50000000    768483     3.0
55000000   1331144     5.0
60000000   1609324     2.0
65000000   1699240     3.0
70000000   1776418     4.0
75000000   1643826     5.0
80000000    932047     4.0
85000000   2292868     4.0
90000000    932191     4.0
95000000   1815101     3.0
100000000   872339     4.0
Movie numpy: [1.000e+00 1.000e+00 1.000e+00 ... 1.777e+04 1.777e+04 1.777e+04]
Length: 100480507
-Dataset examples-
           Cust_Id  Rating  Movie_Id
1          1488844     3.0         1
5000996     501954     2.0       996
10001962    404654     5.0      1962
15002876    886608     2.0      2876
20003825   1193835     2.0      3825
25004661   189

In [3]:
start = time.process_time()
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5}
algo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
#algo = NormalPredictor()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
end = time.process_time()
print("运算时间为"+str(end-start))


Estimating biases using als...
RMSE: 1.0164
Estimating biases using als...
RMSE: 1.0132
Estimating biases using als...
RMSE: 1.0150
运算时间为3.737811999999394


In [8]:
data = open('data.csv', mode='w')
row = list()
file='probe.txt'
with open(file) as f:
    for line in f: 
        del row[:]
        line = line.strip()
        if line.endswith(':'):
    # All below are ratings for this movie, until another movie appears.
            movie_id = line.replace(':', '')
        else:
            row = [x for x in line.split(',')]
            row.insert(0, movie_id)
            data.write(','.join(row))
            data.write('\n')
    #print("Done.\n")
    data.close()

df = pd.read_csv('data.csv', sep=',', names=['movie_id', 'user_id'])
df['predict_rate']=0
df['true_rate']=0


creating the dataframe from data.csv file..


In [45]:
sample_probe=df.sample(frac=0.01, random_state=1)
print('sample的大小为: {}'.format(sample_probe.shape[0]))

sample的大小为: 14084


In [35]:
for i in range(0,sample_probe.shape[0]):
    pred = algo.predict(sample_probe.iloc[i][1], sample_probe.iloc[i][0], r_ui=5, verbose=True)
    sample_probe.iloc[i][2]=pred[3]
    sample_probe.iloc[i][3]=pred[2]

user: 109292     item: 8086       r_ui = 5.00   est = 3.27   {'was_impossible': False}
user: 2178034    item: 963        r_ui = 5.00   est = 3.97   {'was_impossible': False}
user: 1339428    item: 8993       r_ui = 5.00   est = 3.89   {'was_impossible': False}
user: 643502     item: 528        r_ui = 5.00   est = 2.91   {'was_impossible': False}
user: 183052     item: 6724       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 1529030    item: 3864       r_ui = 5.00   est = 4.36   {'was_impossible': False}
user: 1988294    item: 16006      r_ui = 5.00   est = 4.17   {'was_impossible': False}
user: 767537     item: 14272      r_ui = 5.00   est = 3.49   {'was_impossible': False}
user: 884428     item: 6219       r_ui = 5.00   est = 3.75   {'was_impossible': False}
user: 996244     item: 1843       r_ui = 5.00   est = 3.40   {'was_impossible': False}
user: 13070      item: 5496       r_ui = 5.00   est = 3.76   {'was_impossible': False}
user: 2465617    item: 1307       r_ui = 5.

user: 2476145    item: 17174      r_ui = 5.00   est = 3.86   {'was_impossible': False}
user: 2086160    item: 12134      r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 89920      item: 14545      r_ui = 5.00   est = 3.38   {'was_impossible': False}
user: 1452127    item: 1915       r_ui = 5.00   est = 3.57   {'was_impossible': False}
user: 695991     item: 6119       r_ui = 5.00   est = 3.23   {'was_impossible': False}
user: 287545     item: 1317       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 2434858    item: 3825       r_ui = 5.00   est = 3.51   {'was_impossible': False}
user: 1071589    item: 13526      r_ui = 5.00   est = 3.51   {'was_impossible': False}
user: 546543     item: 12694      r_ui = 5.00   est = 3.64   {'was_impossible': False}
user: 1542509    item: 12176      r_ui = 5.00   est = 3.46   {'was_impossible': False}
user: 1371385    item: 4937       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 227919     item: 1470       r_ui = 5.

user: 2040122    item: 1707       r_ui = 5.00   est = 3.70   {'was_impossible': False}
user: 908221     item: 5154       r_ui = 5.00   est = 3.39   {'was_impossible': False}
user: 2007959    item: 6837       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 2602930    item: 1138       r_ui = 5.00   est = 3.02   {'was_impossible': False}
user: 1292299    item: 2482       r_ui = 5.00   est = 3.48   {'was_impossible': False}
user: 891793     item: 11134      r_ui = 5.00   est = 3.81   {'was_impossible': False}
user: 1388584    item: 5034       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 1381825    item: 2391       r_ui = 5.00   est = 3.41   {'was_impossible': False}
user: 90549      item: 10358      r_ui = 5.00   est = 3.09   {'was_impossible': False}
user: 1551218    item: 17107      r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 1682186    item: 8872       r_ui = 5.00   est = 3.90   {'was_impossible': False}
user: 1997031    item: 17149      r_ui = 5.

user: 2615106    item: 1220       r_ui = 5.00   est = 4.01   {'was_impossible': False}
user: 1635010    item: 2122       r_ui = 5.00   est = 3.64   {'was_impossible': False}
user: 1184171    item: 2152       r_ui = 5.00   est = 3.35   {'was_impossible': False}
user: 1231610    item: 10358      r_ui = 5.00   est = 3.10   {'was_impossible': False}
user: 1485331    item: 10451      r_ui = 5.00   est = 3.84   {'was_impossible': False}
user: 549962     item: 4315       r_ui = 5.00   est = 3.56   {'was_impossible': False}
user: 129655     item: 3151       r_ui = 5.00   est = 3.24   {'was_impossible': False}
user: 1466527    item: 6503       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 424825     item: 5516       r_ui = 5.00   est = 3.61   {'was_impossible': False}
user: 2087896    item: 2862       r_ui = 5.00   est = 4.22   {'was_impossible': False}
user: 1228374    item: 2465       r_ui = 5.00   est = 3.86   {'was_impossible': False}
user: 124914     item: 17449      r_ui = 5.

user: 185084     item: 122        r_ui = 5.00   est = 3.74   {'was_impossible': False}
user: 168827     item: 175        r_ui = 5.00   est = 4.01   {'was_impossible': False}
user: 1611809    item: 6014       r_ui = 5.00   est = 3.21   {'was_impossible': False}
user: 2269827    item: 568        r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 986662     item: 1798       r_ui = 5.00   est = 4.06   {'was_impossible': False}
user: 2093196    item: 4577       r_ui = 5.00   est = 3.70   {'was_impossible': False}
user: 263016     item: 227        r_ui = 5.00   est = 3.70   {'was_impossible': False}
user: 2213638    item: 4315       r_ui = 5.00   est = 3.56   {'was_impossible': False}
user: 1262941    item: 252        r_ui = 5.00   est = 3.52   {'was_impossible': False}
user: 1270565    item: 6475       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 2628029    item: 7846       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 2112939    item: 13456      r_ui = 5.

...

user: 692449     item: 9528       r_ui = 5.00   est = 3.91   {'was_impossible': False}
user: 1077968    item: 6528       r_ui = 5.00   est = 3.76   {'was_impossible': False}
user: 1995958    item: 6255       r_ui = 5.00   est = 3.22   {'was_impossible': False}
user: 1446099    item: 6042       r_ui = 5.00   est = 3.18   {'was_impossible': False}
user: 695285     item: 720        r_ui = 5.00   est = 4.10   {'was_impossible': False}
user: 923604     item: 8327       r_ui = 5.00   est = 4.48   {'was_impossible': False}
user: 1100438    item: 10666      r_ui = 5.00   est = 3.75   {'was_impossible': False}
user: 2191134    item: 1409       r_ui = 5.00   est = 3.60   {'was_impossible': False}
user: 246210     item: 3405       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 260992     item: 8993       r_ui = 5.00   est = 4.06   {'was_impossible': False}
user: 2148623    item: 3216       r_ui = 5.00   est = 3.56   {'was_impossible': False}
user: 1086101    item: 13081      r_ui = 5.

user: 1330456    item: 8          r_ui = 5.00   est = 3.51   {'was_impossible': False}
user: 2012886    item: 5401       r_ui = 5.00   est = 3.48   {'was_impossible': False}
user: 1886287    item: 10314      r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 1246336    item: 11812      r_ui = 5.00   est = 4.05   {'was_impossible': False}
user: 2564337    item: 2152       r_ui = 5.00   est = 3.35   {'was_impossible': False}
user: 522622     item: 7944       r_ui = 5.00   est = 3.60   {'was_impossible': False}
user: 348114     item: 3276       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 1758730    item: 5760       r_ui = 5.00   est = 4.30   {'was_impossible': False}
user: 41838      item: 11745      r_ui = 5.00   est = 3.08   {'was_impossible': False}
user: 828011     item: 4338       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 909640     item: 5991       r_ui = 5.00   est = 3.84   {'was_impossible': False}
user: 2110943    item: 11691      r_ui = 5.

user: 1740952    item: 3333       r_ui = 5.00   est = 3.27   {'was_impossible': False}
user: 577740     item: 13081      r_ui = 5.00   est = 3.53   {'was_impossible': False}
user: 1106637    item: 11077      r_ui = 5.00   est = 3.61   {'was_impossible': False}
user: 2314817    item: 494        r_ui = 5.00   est = 4.00   {'was_impossible': False}
user: 2613751    item: 2016       r_ui = 5.00   est = 3.93   {'was_impossible': False}
user: 735984     item: 361        r_ui = 5.00   est = 3.55   {'was_impossible': False}
user: 93738      item: 3418       r_ui = 5.00   est = 3.32   {'was_impossible': False}
user: 1108505    item: 406        r_ui = 5.00   est = 3.52   {'was_impossible': False}
user: 1374073    item: 5092       r_ui = 5.00   est = 4.45   {'was_impossible': False}
user: 1884264    item: 1145       r_ui = 5.00   est = 2.99   {'was_impossible': False}
user: 1598164    item: 11134      r_ui = 5.00   est = 3.81   {'was_impossible': False}
user: 2026943    item: 4951       r_ui = 5.

user: 74765      item: 17053      r_ui = 5.00   est = 3.49   {'was_impossible': False}
user: 491203     item: 2012       r_ui = 5.00   est = 3.36   {'was_impossible': False}
user: 1575885    item: 13439      r_ui = 5.00   est = 3.73   {'was_impossible': False}
user: 1390833    item: 15449      r_ui = 5.00   est = 3.44   {'was_impossible': False}
user: 2326833    item: 14890      r_ui = 5.00   est = 3.21   {'was_impossible': False}
user: 167187     item: 1798       r_ui = 5.00   est = 4.06   {'was_impossible': False}
user: 714559     item: 311        r_ui = 5.00   est = 3.77   {'was_impossible': False}
user: 2549716    item: 200        r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 388034     item: 5926       r_ui = 5.00   est = 4.09   {'was_impossible': False}
user: 1322996    item: 1642       r_ui = 5.00   est = 3.85   {'was_impossible': False}
user: 1441464    item: 1682       r_ui = 5.00   est = 3.94   {'was_impossible': False}
user: 1463399    item: 313        r_ui = 5.

user: 1093594    item: 5327       r_ui = 5.00   est = 3.73   {'was_impossible': False}
user: 458450     item: 4123       r_ui = 5.00   est = 3.53   {'was_impossible': False}
user: 191905     item: 11607      r_ui = 5.00   est = 4.23   {'was_impossible': False}
user: 1028413    item: 3864       r_ui = 5.00   est = 4.29   {'was_impossible': False}
user: 717190     item: 2152       r_ui = 5.00   est = 3.27   {'was_impossible': False}
user: 372228     item: 3938       r_ui = 5.00   est = 3.97   {'was_impossible': False}
user: 2222111    item: 357        r_ui = 5.00   est = 3.17   {'was_impossible': False}
user: 738665     item: 6534       r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 1576295    item: 6972       r_ui = 5.00   est = 3.53   {'was_impossible': False}
user: 1577424    item: 4640       r_ui = 5.00   est = 4.07   {'was_impossible': False}
user: 432057     item: 636        r_ui = 5.00   est = 3.29   {'was_impossible': False}
user: 2010845    item: 3239       r_ui = 5.

In [44]:
#print(algo.best_score['rmse'])
from math import sqrt
from sklearn.metrics import mean_squared_error
print('RMSE误差值为: {}'.format(mean_squared_error(sample_probe.iloc[:,2], sample_probe.iloc[:,3])))

RMSE误差值为: 3.6972451008236296
